In [1]:
import cv2
import numpy as np
import random
import math

In [4]:
img=cv2.imread('../file/image0.jpg',1)
imgInfo=img.shape                           #(547, 730, 3)
height=imgInfo[0]
width=imgInfo[1]
mode=imgInfo[2]                             #每个像素点是由三个基本颜色颜色组成的

###### 灰度处理

In [ ]:
'''方法1 通过imread实现图片灰度处理'''
img0=cv2.imread('../file/image0.jpg',0)     #设置为0就是灰度图片
img1=cv2.imread('../file/image0.jpg',1)
print(img0.shape)
print(img1.shape)
cv2.imshow('src',img0)
cv2.waitKey(0)

(547, 730)
(547, 730, 3)


In [ ]:
'''方法2 调用cvtColor api实现图片灰度处理'''
dst=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)     #使用cvtColor将彩色图片转换为灰度图片
cv2.imshow('dst',dst)
cv2.waitKey(0)

In [ ]:
'''方法3 源码实现'''
'''原理：如果灰度图像也是rgb类型 那么R=G=B    灰度处理公式：(R+G+B)/3'''
dst=np.zeros((height,width,3),np.uint8)
for i in range(0,height):
    for j in range(0,width):
        (b,g,r)=img[i,j]
        gray=(int(b)+int(g)+int(r))/3
        dst[i,j]=np.uint8(gray)
cv2.imshow('dst',dst)
cv2.waitKey(0)

In [ ]:
'''方法4 源码实现'''
'''原理：心理学计算公式    灰度处理公式：gray=R*0.299+G*0.587+B*0.114'''
dst=np.zeros((height,width,3),np.uint8)
for i in range(0,height):
    for j in range(0,width):
        (b,g,r)=img[i,j]
        b=int(b)
        g=int(g)
        b=int(b)
        #gray=r*0.299+g*0.587+b*0.114        #算法优化之前
        #gray=(r*1+g*2+b*1)/4                #算法优化之后
        gray=(r+(g<<1)+b)>>2                 #算法优化2.0
        dst[i,j]=np.uint8(gray)h
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 颜色反转

In [ ]:
'''公式：255-当前像素值'''
'''源码实现：灰度图片颜色反转'''
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst=np.zeros((height,width,1),np.uint8)             #1表示灰度图片
for i in range(0,height):
    for j in range(0,width):
        grayPixel=gray[i,j]
        dst[i,j]=255-grayPixel
cv2.imshow('dst',dst)
cv2.waitKey(0)

In [ ]:
'''源码实现：彩色图片颜色反转'''
dst=np.zeros((height,width,3),np.uint8)             #1表示灰度图片
for i in range(0,height):
    for j in range(0,width):
        (b,g,r)=img[i,j]
        dst[i,j]=(255-b,255-g,255-r)
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 马赛克

In [ ]:
'''原理：挑选一个像素点，10*10的范围内的像素点都和这个相同'''
'''源码实现：'''
for m in range(100,300):
    for n in range(100,200):
        if m%10==0 and n%10==0:          #每隔10个宽高的矩形框选取一个像素
            for i in range(0,10):
                for j in range(0,10):
                    (b,g,r)=img[m,n]
                    img[i+m,j+n]=(b,g,r)
cv2.imshow('dst',img)
cv2.waitKey(0)

###### 毛玻璃

In [ ]:
'''原理：每个像素点周围随机的一个点来替换掉当前的像素点'''
'''源码实现：'''
dst=np.zeros((height,width,3),np.uint8)
mm=8                                             #取8*8范围内的随机像素点
for m in range(0,height-mm):                     #防止最后一个点取不到+8的像素点
    for n in range(0,width-mm):
        index=int(random.random()*mm)            #0-8之间随机数
        (b,g,r)=img[m+index,n+index]             #选择像素点周围[0-8,0-8]随机的一个像素点
        dst[m,n]=(b,g,r)
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 图片融合

In [ ]:
'''调用addWeighted api实现图片融合'''
'''原理：公式：  dst：目标图片  src：原图片   dst=src1*a+src2*(1-a)'''
img0=cv2.imread('../file/image0.jpg',1)
img1=cv2.imread('../file/image1.jpg',1)
imgInfo=img0.shape
height=imgInfo[0]
width=imgInfo[1]
'''ROI：感兴趣的范围(高宽要比img0和img1都要小)'''
roiH=int(height/2)
roiW=int(width/2)
img0ROI=img0[0:roiH,0:roiW]           #img0选取的区域
img1ROI=img1[0:roiH,0:roiW]           #img1选取的区域
dst=np.zeros((roiH,roiW,3),np.uint8)
dst=cv2.addWeighted(img0ROI,0.5,img1ROI,0.5,0)   #两张图片融合 两个0.5分别对应每张图片的权重 0暂时不关心
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 边缘检测

In [ ]:
'''方法1：调用Canny api实现边缘检测'''
'''过程：1.灰度处理 2.高斯滤波 3.调用canny方法'''
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)      #灰度处理
imgG=cv2.GaussianBlur(gray,(3,3),0)            #高斯滤波处理 (3,3),0:模板大小 先暂定是这样
'''50,50：门限   如果图片经过卷积之后的值大于门限，就认为是边缘点 '''
dst=cv2.Canny(img,50,50)
cv2.imshow('dst',dst)
cv2.waitKey(0)

In [ ]:
'''方法2：源码实现'''
# sobel 1 算子模版 2 图片卷积 3 阈值判决 
# [1 2 1          [ 1 0 -1
#  0 0 0            2 0 -2
# -1 -2 -1 ]       1 0 -1 ]
              
# [1 2 3 4] [a b c d] a*1+b*2+c*3+d*4 = dst
# sqrt(a*a+b*b) = f>th
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst = np.zeros((height,width,1),np.uint8)
for i in range(0,height-2):
    for j in range(0,width-2):
        gy = gray[i,j]*1+gray[i,j+1]*2+gray[i,j+2]*1-gray[i+2,j]*1-gray[i+2,j+1]*2-gray[i+2,j+2]*1
        gx = gray[i,j]+gray[i+1,j]*2+gray[i+2,j]-gray[i,j+2]-gray[i+1,j+2]*2-gray[i+2,j+2]
        grad = math.sqrt(gx*gx+gy*gy)
        if grad>50:
            dst[i,j] = 255
        else:
            dst[i,j] = 0
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 浮雕效果

In [ ]:
'''公式：新的像素值=相邻像素值之差+150(数值非固定)'''
'''源码实现：'''
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst=np.zeros((height,width,1),np.uint8)           #1 新图片为灰度图片
for i in range(0,height):
    for j in range(0,width-1):
        grayP0=int(gray[i,j])
        grayP1=int(gray[i,j+1])
        newP=grayP0-grayP1+150
        if newP > 255:
            newP=255
        if newP < 0:
            newP=0
        dst[i,j]=newP
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 颜色风格

In [ ]:
'''公式(自定义)：b=b*1.5 g=g*1.3'''
'''源码实现：'''
dst=np.zeros((height,width,3),np.uint8)           #1 新图片为彩色图片
for i in range(0,height):
    for j in range(0,width):
        (b,g,r)=img[i,j]
        b=b*1.5
        g=g*1.3
        if b>255:
            b=255
        if g>255:
            g=255
        dst[i,j]=(b,g,r)
cv2.imshow('dst',dst)
cv2.waitKey(0)

###### 油画特效

In [ ]:
'''过程：
1.灰度处理 2.将图片分割成若干个小方块，统计这些小方块中每一个像素的灰度值 
3.将0-255的灰度值划分成几个等级，并把第二步处理的结果映射到这个范围内
4.找到每个方块中灰度等级最多的所有的像素，求取这些像素的均值
5.用统计出来的平均值来替代原来的像素值
'''
'''源码实现：'''
img=cv2.imread('../file/image00.jpg',1)
imgInfo=img.shape
height=imgInfo[0]
width=imgInfo[1]
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst=np.zeros((height,width,3),np.uint8)
for i in range(4,height-4):
    for j in range(4,width-4):
        array1=np.zeros(8,np.uint8)
        for m in range(-4,4):          #8*8的小方格
            for n in range(-4,4):
                p1=int(gray[i+m,j+n]/32)
                array1[p1]=array1[p1]+1
        currentMax=array1[0]
        l=0
        for k in range(0,8):
            if currentMax<array1[k]:
                currentMax=array1[k]
                l=k
        for m in range(-4,4):           #简化方法求均值
            for n in range(-4,4):
                if gray[i+m,j+n]>=(l*32) and gray[i+m,j+n]<=((l+1)*32):
                    (b,g,r)=img[i+m,j+n]
        dst[i,j]=(b,g,r)
cv2.imshow('dst',dst)
cv2.waitKey(0)        

###### 线段绘制

In [ ]:
newImageInfo=(500,500,3)                             #彩色图片
dst=np.zeros(newImageInfo,np.uint8)
'''绘制线段：api:line
参数： 2.起始位置 3.终止位置 4.颜色 5.线段的宽度 6.线段的类型
'''
cv2.line(dst,(100,100),(400,400),(0,0,255))
cv2.line(dst,(100,200),(400,200),(0,255,255),20)
cv2.line(dst,(100,300),(400,300),(0,255,0),20,cv2.LINE_AA)

cv2.line(dst,(200,150),(50,250),(25,100,255))       #绘制一个三角形
cv2.line(dst,(50,250),(400,380),(255,100,255))
cv2.line(dst,(400,380),(200,150),(255,100,255))
cv2.imshow('dst',dst)
cv2.waitKey(0) 

###### 矩形圆形任意多边形绘制

In [ ]:
newImageInfo=(500,500,3)
dst=np.zeros(newImageInfo,np.uint8)
'''绘制矩形：api:rectangle
参数： 2.矩形左上角 3.矩形右下角 4.颜色 5.是否填充(-1：表示填充 >0：线条的宽度)'''
cv2.rectangle(dst,(50,100),(200,300),(255,0,0),-1)
'''绘制圆形：api:circle
参数： 2.圆心 3.半径 4.颜色 5.是否填充(-1：表示填充 >0：线条的宽度)'''
cv2.circle(dst,(250,250),(50),(0,255,0),2)
'''绘制椭圆圆弧：api:ellipse
参数： 2.圆心 3.长轴短轴 4.偏转角度 5.圆弧起始角度 6.圆弧终止角度 7.颜色 8.是否填充(-1：表示填充 >0：线条的宽度)'''
cv2.ellipse(dst,(256,256),(150,100),0,0,180,(0,255,0),-1)
'''绘制任意多边形：api:polylines'''
points=np.array([[150,50],[140,140],[200,170],[250,250],[150,50]],np.int32)
print(points.shape)
points=points.reshape((-1,1,2))         #实现了维度转换
print(points.shape)
cv2.polylines(dst,[points],True,(0,255,255))        
cv2.imshow('dst',dst)
cv2.waitKey(0) 

(5, 2)
(5, 1, 2)


###### 文字图片绘制

In [ ]:
'''绘制文字：api:putText
参数： 2.文字内容 3.起始位置 4.字体 5.字体大小 6.颜色 7.粗细 8.线条类型'''
font=cv2.FONT_HERSHEY_SIMPLEX                           #定义一个简单文字
cv2.rectangle(img,(200,100),(500,400),(0,255,0),3)      #定义一个矩形
cv2.putText(img,'this is flower',(100,300),font,1,(200,100,255),2,cv2.LINE_AA)
cv2.imshow('src',img)
cv2.waitKey(0) 

In [ ]:
'''图片绘制'''
height=int(img.shape[0]*0.2)
width=int(img.shape[1]*0.2)
imgResize=cv2.resize(img,(width,height))            #resize实现图像的等比例缩放
for i in range(0,height):
    for j in range(0,width):
        img[i+200,j+350]=imgResize[i,j]
cv2.imshow('src',img)
cv2.waitKey(0) 